In [10]:
import os
import re
from collections import Counter

import cv2
import numpy as np
import open3d as o3d

from oct_point_cloud import (create_mesh_sphere, create_point_cloud_from_vol,
                             find_lowest_point, get_depth_map,
                             get_largest_cluster, get_points_and_colors,
                             inpaint_layers, needle_cloud_find_needle_tip,
                             remove_outliers, calculate_needle_tip_depth)

In [3]:
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]  

segmented_images_path = '/home/demir/Desktop/jhu_project/needle-segmentation/data/test_seg_res'
oct_images_path = '/home/demir/Desktop/jhu_project/oct_scans/jun11/2.1/images'

num_b_scans_volume = 5
target_depth = 0.5

seg_masks = [(cv2.imread(os.path.join(segmented_images_path, img), cv2.IMREAD_UNCHANGED)) for img in sorted(os.listdir(segmented_images_path), key=natural_sort_key)]
oct_images = [cv2.imread(os.path.join(oct_images_path, img), cv2.IMREAD_GRAYSCALE) for img in sorted(os.listdir(oct_images_path), key=natural_sort_key)]

In [4]:
seg_idx = 8
seg_volume = np.stack(seg_masks[seg_idx*num_b_scans_volume:(seg_idx+1)*num_b_scans_volume], axis=0)
print(seg_volume.shape)
print(np.unique(seg_volume))

(5, 1024, 1024)
[0 1 2 3]


In [5]:
needle_point_cloud = create_point_cloud_from_vol(seg_volume, seg_index=[1])

needle_tip_coords, cleaned_needle_point_cloud = needle_cloud_find_needle_tip(needle_point_cloud, return_clean_point_cloud=True)

# inpaint layers
ilm_depth_map = get_depth_map(seg_volume, seg_index=2)
rpe_depth_map = get_depth_map(seg_volume, seg_index=3)

assert ilm_depth_map.shape == rpe_depth_map.shape

ilm_points, rpe_points = inpaint_layers(ilm_depth_map, rpe_depth_map)

ilm_colors = np.array([[0, 1, 0] for _ in range(ilm_points.shape[0])])
rpe_colors = np.array([[0, 0, 1] for _ in range(rpe_points.shape[0])])

# create layers point cloud
layers_point_cloud = o3d.geometry.PointCloud()
layers_point_cloud.points = o3d.utility.Vector3dVector(np.vstack((ilm_points, rpe_points)))
layers_point_cloud.colors = o3d.utility.Vector3dVector(np.vstack((ilm_colors, rpe_colors)))

print('needle tip coords:', needle_tip_coords)

needle tip coords: [  0. 361. 755.]


In [6]:
# find in the new 3D volume where the pixels corresponding to ilm and rpe are at the needle tip
ilm_tip_coords = ilm_points[(ilm_points[:, 0] == needle_tip_coords[0]) & (ilm_points[:, 2] == needle_tip_coords[2])]
rpe_tip_coords = rpe_points[(rpe_points[:, 0] == needle_tip_coords[0]) & (rpe_points[:, 2] == needle_tip_coords[2])]

print(ilm_tip_coords, rpe_tip_coords)

ilm_sphere = create_mesh_sphere(ilm_tip_coords[0], radius=2, color=[0,0,0])
rpe_sphere = create_mesh_sphere(rpe_tip_coords[0], radius=2, color=[0,0,0])

[[  0. 291. 755.]] [[  0.         395.00012207 755.        ]]


In [11]:
_, _, needle_tip_depth_relative_percentage = calculate_needle_tip_depth(needle_tip_coords, ilm_tip_coords[0], rpe_tip_coords[0])
print('needle tip depth relative percentage:', needle_tip_depth_relative_percentage)

needle tip depth relative percentage: 0.6730761330517895


In [12]:
# create needle tip annotation
needle_tip_sphere = create_mesh_sphere(needle_tip_coords)

# add cylinder goind through needle tip intersecting ilm and rpe
ascan_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=0.3, height=500)
transform = np.array([
    [1, 0, 0, needle_tip_coords[0]],
    [0, 0, 1, needle_tip_coords[1]],
    [0, -1, 0, needle_tip_coords[2]],
    [0, 0, 0, 1]
])
ascan_cylinder.transform(transform)

TriangleMesh with 102 points and 200 triangles.

In [13]:
vis = o3d.visualization.Visualizer()

vis.create_window()
vis.add_geometry(cleaned_needle_point_cloud)
vis.add_geometry(layers_point_cloud)

vis.add_geometry(needle_tip_sphere)
vis.add_geometry(ilm_sphere)
vis.add_geometry(rpe_sphere)
vis.add_geometry(ascan_cylinder)
vis.run()
vis.destroy_window()

In [197]:
# Unchanged point cloud
original_needle_pc = create_point_cloud_from_vol(seg_volume, seg_index=[1])
original_ilm_pc = create_point_cloud_from_vol(seg_volume, seg_index=[2])
original_rpe_pc = create_point_cloud_from_vol(seg_volume, seg_index=[3])

noise_removed = remove_outliers(original_needle_pc, nb_points=5, radius=4)

labels = np.array(
        noise_removed.cluster_dbscan(eps=5, min_points=10, print_progress=True))

import matplotlib.pyplot as plt

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
noise_removed.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([noise_removed])

# o3d.visualization.draw_geometries([original_needle_pc])


point cloud has 5 clustersPrecompute neighbors.[=>                                      ] 2%
Precompute neighbors.[========================================] 100%
